In [1]:
import numpy as np
from collections import Counter
from tqdm import tqdm_notebook

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

from utils import *
from data_utils import Vocabulary
from train_utils import train
import random
import pickle

In [2]:
def tokenize_corpus(corpus_path):
    num_lines = len(open(corpus_path, encoding='utf8').read().split('\n'))
    
    # tokenize corpus
    output = ''
    with open(corpus_path, encoding='utf8') as f:
        for line in tqdm_notebook(f, desc='Tokenizing', total=num_lines):
            tokens = tokenizer(line.strip()) + ['\n']
            output += ' '.join(tokens)
    
    # save tokenized corpus
    tok_corpus_path = corpus_path[:-4] + '_tok.txt'
    with open(tok_corpus_path, 'w', encoding='utf8') as f:
        f.write(output)

In [3]:
train_data = 'moulavi_norm.txt'
train_data_tok = train_data[:-4] + '_tok.txt'

In [4]:
tokenize_corpus(train_data)

text = open(train_data, encoding='utf8').read().split('\n')[:10]
print(text)

print('\n\nAfter Tokenizing:\n\n')
print(tokenizer('\n'.join(text)))


['', '  \t', 'بشنو این نی چون شکایت می کند', 'از جداییها حکایت می کند', 'کز نیستان تا مرا ببریده اند', 'در نفیرم مرد و زن نالیده اند', 'سینه خواهم شرحه شرحه از فراق', 'تا بگویم شرح درد اشتیاق', 'هر کسی کو دور ماند از اصل خویش', 'باز جوید روزگار وصل خویش']


After Tokenizing:


[' \t ', 'بشنو', 'این', 'نی', 'چون', 'شکایت', 'می', 'کند', 'از', 'جداییها', 'حکایت', 'می', 'کند', 'کز', 'نیستان', 'تا', 'مرا', 'ببریده', 'اند', 'در', 'نفیرم', 'مرد', 'و', 'زن', 'نالیده', 'اند', 'سینه', 'خواهم', 'شرحه', 'شرحه', 'از', 'فراق', 'تا', 'بگویم', 'شرح', 'درد', 'اشتیاق', 'هر', 'کسی', 'کو', 'دور', 'ماند', 'از', 'اصل', 'خویش', 'باز', 'جوید', 'روزگار', 'وصل', 'خویش']


In [5]:
class Corpus(object):
    
    def __init__(self, corpus_path):
        self.vocabulary = Vocabulary()
        self.corpus_path = corpus_path
        self.num_sentences = len([line for line in open(corpus_path, encoding='utf8')])
    
    def get_data(self, max_vocab=30000, min_count=3, batch_size=20, split_ratio=0.2):
        
        # First pass: add words to the vocabulary
        trn_tokens, val_tokens = [], []
        with open(self.corpus_path, encoding='utf8') as f:
            for line in tqdm_notebook(f, desc='Building Vocab...', total=self.num_sentences):
                tokens = line.split() + ['<EOS>']
                if len(line) <= 10: continue
                if random.random() < split_ratio:
                    val_tokens += tokens
                else:
                    trn_tokens += tokens
        
        counter = Counter(trn_tokens + val_tokens)
        
        vocabs = [(w, c) for (w, c) in counter.most_common(max_vocab) if c >= min_count]
        
        for i, (word, count) in enumerate(vocabs):
            self.vocabulary.word2index[word] = i
            self.vocabulary.word2count[word] = count
            self.vocabulary.index2word[i] = word
            self.vocabulary.num_words += 1
        self.vocabulary.add_word('<UNK>')
        
        
        UNK_TOKEN = self.vocabulary.word2index['<UNK>']
        
        # train ids
        trn_ids = torch.LongTensor(len(trn_tokens))
        for idx, token in enumerate(trn_tokens):
            if token in self.vocabulary.word2index:
                trn_ids[idx] = self.vocabulary.word2index[token] 
            else:
                trn_ids[idx] = UNK_TOKEN
        
        val_ids = torch.LongTensor(len(val_tokens))
        for idx, token in enumerate(val_tokens):
            if token in self.vocabulary.word2index:
                val_ids[idx] = self.vocabulary.word2index[token] 
            else:
                val_ids[idx] = UNK_TOKEN
        
        num_batches = trn_ids.size(0) // batch_size
        trn_ids = trn_ids[: num_batches * batch_size]
        
        num_batches = val_ids.size(0) // batch_size
        val_ids = trn_ids[: num_batches * batch_size]

        return trn_ids.view(batch_size, -1), val_ids.view(batch_size, -1)

In [6]:
max_vocab = 30000
min_count = 1

# LSTM hyper-parameters
embed_size = 1500
hidden_size = 1500
num_layers = 2

# Training hyper-parameters
num_epochs = 40
batch_size = 50
seq_length = 60
learning_rate = 0.001

In [7]:
corpus = Corpus(train_data_tok)
trn_ids, val_ids = corpus.get_data(max_vocab, min_count, batch_size)
vocab_size = len(corpus.vocabulary)

# save vocabs and ids
pickle.dump(corpus.vocabulary, open('vocab.pkl', 'wb'))
np.save('trn_ids.npy', trn_ids.view(-1).numpy())
np.save('val_ids.npy', val_ids.view(-1).numpy())

In [8]:
vocab_size = len(corpus.vocabulary)
print(vocab_size)
print(trn_ids.size())
print(val_ids.size())

25175
torch.Size([50, 6341])
torch.Size([50, 1571])


In [9]:
most_commons = [(w, c) for (w, c) in corpus.vocabulary.word2count.items()][:20]

for w, c in most_commons:
    print(w, c)

<EOS> 52956
و 12871
از 7709
را 6764
در 6362
که 6355
آن 5799
تو 4578
می 4078
او 3878
بر 3873
ای 3650
این 3496
چون 3398
ز 3286
به 2848
تا 2847
من 2821
بود 2163
هر 2092


In [10]:
class LSTM_LM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1, drop=0.35, tie=True):
        super(LSTM_LM, self).__init__()
        
        if tie:
            embed_size = hidden_size
        
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
            
        self.dropout = nn.Dropout(drop)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, dropout=0.35)
        self.fc = nn.Linear(hidden_size, vocab_size)
        
        if tie:
            # Use the same weights both for embedding and classification
            self.fc.weight.data = self.embedding.weight.data
            
        self.init_weights()
        
    def init_weights(self):
        self.embedding.weight.data.uniform_(-0.1, 0.1)
        self.fc.weight.data.uniform_(-0.1, 0.1)
        self.fc.bias.data.fill_(0)
        
    def init_hidden(self, batch_size):
        return (to_var(torch.zeros(self.num_layers, batch_size, self.hidden_size)),
                to_var(torch.zeros(self.num_layers, batch_size, self.hidden_size)))
        
    def forward(self, x, hidden):
        # embed word ids to vectors
        x = self.embedding(x)
        x = self.dropout(x)  # DROPOUT
        
        # forward RNN step
        x, hidden = self.lstm(x, hidden)
        x = self.dropout(x)  # DROPOUT
        
        # reshape output to (bs * seq_length, hidden_size)
        x = x.contiguous().view(x.size(0) * x.size(1), x.size(2))
        
        # decode hidden states of all time steps
        x = self.fc(x)
        
        return x, hidden
    
    def save(self, epoch, loss):
        filename = 'lm-masnavi-epoch-{}-em-{}-hi-{}-nl-{}-{:.2f}-{:.2f}.pth'.format(
            epoch, self.embed_size, self.hidden_size, self.num_layers, loss, np.exp(loss))
        torch.save(self.state_dict(), filename)

In [11]:
torch.cuda.init()
# # model
model = LSTM_LM(vocab_size, embed_size, hidden_size, num_layers, drop=0.65).cuda()

# loss function
criterion = nn.CrossEntropyLoss().cuda()

    
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [12]:
hist = train(model, trn_ids, val_ids, 
             criterion, optimizer, scheduler, 
             num_epochs, batch_size, seq_length)

/home/mamarez/Language_model/utils.py:13: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(x, volatile=volatile)


here
here


/home/mamarez/Language_model/train_utils.py:38: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(), 0.3)


Training: Epoch [1/40], Step [1/105], Loss: 10.131, Perp: 25106.65, Acc: 0.00           here
here
Training: Epoch [1/40], Step [2/105], Loss: 10.128, Perp: 25027.48, Acc: 0.00           here
here
Training: Epoch [1/40], Step [3/105], Loss: 10.159, Perp: 25811.83, Acc: 0.00           here
here
Training: Epoch [1/40], Step [4/105], Loss: 10.155, Perp: 25719.59, Acc: 0.00           here
here
Training: Epoch [1/40], Step [5/105], Loss: 10.151, Perp: 25607.64, Acc: 0.00           here
here
Training: Epoch [1/40], Step [6/105], Loss: 10.145, Perp: 25462.66, Acc: 0.00           here
here
Training: Epoch [1/40], Step [7/105], Loss: 10.139, Perp: 25311.25, Acc: 0.01           here
here
Training: Epoch [1/40], Step [8/105], Loss: 10.134, Perp: 25183.54, Acc: 0.01           here
here
Training: Epoch [1/40], Step [9/105], Loss: 10.129, Perp: 25067.94, Acc: 0.01           here
here
Training: Epoch [1/40], Step [10/105], Loss: 10.125, Perp: 24954.01, Acc: 0.01           here
here
Training: Epoch [1/

Training: Epoch [1/40], Step [84/105], Loss: 10.008, Perp: 22198.44, Acc: 0.01           here
here
Training: Epoch [1/40], Step [85/105], Loss: 10.007, Perp: 22182.97, Acc: 0.01           here
here
Training: Epoch [1/40], Step [86/105], Loss: 10.006, Perp: 22167.47, Acc: 0.01           here
here
Training: Epoch [1/40], Step [87/105], Loss: 10.006, Perp: 22151.81, Acc: 0.01           here
here
Training: Epoch [1/40], Step [88/105], Loss: 10.005, Perp: 22137.17, Acc: 0.01           here
here
Training: Epoch [1/40], Step [89/105], Loss: 10.004, Perp: 22122.55, Acc: 0.01           here
here
Training: Epoch [1/40], Step [90/105], Loss: 10.004, Perp: 22107.59, Acc: 0.01           here
here
Training: Epoch [1/40], Step [91/105], Loss: 10.003, Perp: 22093.02, Acc: 0.01           here
here
Training: Epoch [1/40], Step [92/105], Loss: 10.002, Perp: 22078.53, Acc: 0.01           here
here
Training: Epoch [1/40], Step [93/105], Loss: 10.002, Perp: 22064.41, Acc: 0.01           here
here
Training: 

...

Training: Epoch [37/40], Step [98/105], Loss: 2.007, Perp: 7.44, Acc: 0.56           here
here
Training: Epoch [37/40], Step [99/105], Loss: 2.007, Perp: 7.44, Acc: 0.56           here
here
Training: Epoch [37/40], Step [100/105], Loss: 2.007, Perp: 7.44, Acc: 0.56           here
here
Training: Epoch [37/40], Step [101/105], Loss: 2.007, Perp: 7.44, Acc: 0.56           here
here
Training: Epoch [37/40], Step [102/105], Loss: 2.007, Perp: 7.44, Acc: 0.56           here
here
Training: Epoch [37/40], Step [103/105], Loss: 2.007, Perp: 7.44, Acc: 0.56           here
here
Training: Epoch [37/40], Step [104/105], Loss: 2.007, Perp: 7.44, Acc: 0.56           here
here
Training: Epoch [37/40], Step [105/105], Loss: 2.007, Perp: 7.44, Acc: 0.56           here
here
Validation: Epoch [37/40], Step [1/26], Loss: 1.712, Perp: 5.54, Acc: 0.67           here
here
Validation: Epoch [37/40], Step [2/26], Loss: 1.705, Perp: 5.50, Acc: 0.67           here
here
Validation: Epoch [37/40], Step [3/26], Loss

Training: Epoch [38/40], Step [54/105], Loss: 1.953, Perp: 7.05, Acc: 0.56           here
here
Training: Epoch [38/40], Step [55/105], Loss: 1.953, Perp: 7.05, Acc: 0.56           here
here
Training: Epoch [38/40], Step [56/105], Loss: 1.953, Perp: 7.05, Acc: 0.56           here
here
Training: Epoch [38/40], Step [57/105], Loss: 1.953, Perp: 7.05, Acc: 0.56           here
here
Training: Epoch [38/40], Step [58/105], Loss: 1.953, Perp: 7.05, Acc: 0.56           here
here
Training: Epoch [38/40], Step [59/105], Loss: 1.953, Perp: 7.05, Acc: 0.56           here
here
Training: Epoch [38/40], Step [60/105], Loss: 1.953, Perp: 7.05, Acc: 0.56           here
here
Training: Epoch [38/40], Step [61/105], Loss: 1.953, Perp: 7.05, Acc: 0.56           here
here
Training: Epoch [38/40], Step [62/105], Loss: 1.953, Perp: 7.05, Acc: 0.56           here
here
Training: Epoch [38/40], Step [63/105], Loss: 1.953, Perp: 7.05, Acc: 0.56           here
here
Training: Epoch [38/40], Step [64/105], Loss: 1.95

Training: Epoch [39/40], Step [10/105], Loss: 1.893, Perp: 6.64, Acc: 0.58           here
here
Training: Epoch [39/40], Step [11/105], Loss: 1.892, Perp: 6.64, Acc: 0.58           here
here
Training: Epoch [39/40], Step [12/105], Loss: 1.892, Perp: 6.63, Acc: 0.58           here
here
Training: Epoch [39/40], Step [13/105], Loss: 1.892, Perp: 6.63, Acc: 0.58           here
here
Training: Epoch [39/40], Step [14/105], Loss: 1.892, Perp: 6.63, Acc: 0.58           here
here
Training: Epoch [39/40], Step [15/105], Loss: 1.892, Perp: 6.63, Acc: 0.58           here
here
Training: Epoch [39/40], Step [16/105], Loss: 1.892, Perp: 6.63, Acc: 0.58           here
here
Training: Epoch [39/40], Step [17/105], Loss: 1.891, Perp: 6.63, Acc: 0.58           here
here
Training: Epoch [39/40], Step [18/105], Loss: 1.891, Perp: 6.63, Acc: 0.58           here
here
Training: Epoch [39/40], Step [19/105], Loss: 1.891, Perp: 6.63, Acc: 0.58           here
here
Training: Epoch [39/40], Step [20/105], Loss: 1.89

Training: Epoch [39/40], Step [97/105], Loss: 1.887, Perp: 6.60, Acc: 0.58           here
here
Training: Epoch [39/40], Step [98/105], Loss: 1.887, Perp: 6.60, Acc: 0.58           here
here
Training: Epoch [39/40], Step [99/105], Loss: 1.886, Perp: 6.60, Acc: 0.58           here
here
Training: Epoch [39/40], Step [100/105], Loss: 1.886, Perp: 6.60, Acc: 0.58           here
here
Training: Epoch [39/40], Step [101/105], Loss: 1.886, Perp: 6.60, Acc: 0.58           here
here
Training: Epoch [39/40], Step [102/105], Loss: 1.886, Perp: 6.60, Acc: 0.58           here
here
Training: Epoch [39/40], Step [103/105], Loss: 1.886, Perp: 6.60, Acc: 0.58           here
here
Training: Epoch [39/40], Step [104/105], Loss: 1.886, Perp: 6.59, Acc: 0.58           here
here
Training: Epoch [39/40], Step [105/105], Loss: 1.886, Perp: 6.59, Acc: 0.58           here
here
Validation: Epoch [39/40], Step [1/26], Loss: 1.565, Perp: 4.78, Acc: 0.72           here
here
Validation: Epoch [39/40], Step [2/26], Loss

Training: Epoch [40/40], Step [53/105], Loss: 1.843, Perp: 6.32, Acc: 0.59           here
here
Training: Epoch [40/40], Step [54/105], Loss: 1.843, Perp: 6.32, Acc: 0.59           here
here
Training: Epoch [40/40], Step [55/105], Loss: 1.843, Perp: 6.32, Acc: 0.59           here
here
Training: Epoch [40/40], Step [56/105], Loss: 1.843, Perp: 6.32, Acc: 0.59           here
here
Training: Epoch [40/40], Step [57/105], Loss: 1.843, Perp: 6.32, Acc: 0.59           here
here
Training: Epoch [40/40], Step [58/105], Loss: 1.843, Perp: 6.32, Acc: 0.59           here
here
Training: Epoch [40/40], Step [59/105], Loss: 1.843, Perp: 6.32, Acc: 0.59           here
here
Training: Epoch [40/40], Step [60/105], Loss: 1.843, Perp: 6.32, Acc: 0.59           here
here
Training: Epoch [40/40], Step [61/105], Loss: 1.843, Perp: 6.32, Acc: 0.59           here
here
Training: Epoch [40/40], Step [62/105], Loss: 1.843, Perp: 6.32, Acc: 0.59           here
here
Training: Epoch [40/40], Step [63/105], Loss: 1.84

In [14]:
torch.save(model.state_dict(), 'masnavi-lm.pth')

In [43]:
N = 1000
p = torch.FloatTensor([0.5, 0.25, 0.15, 0.10])
p = torch.FloatTensor([50, 25, 15, 10])

counter = Counter()
# Draw N samples
for _ in range(N):
    sample = torch.multinomial(p, num_samples=1, replacement=True).numpy()[0]
    counter[sample] += 1

for sample, count in counter.most_common():
    print("{:d}: {:2d}".format(sample, count))

0: 494
1: 240
2: 170
3: 96


In [44]:
def get_sample(model, sample_len):
    model.eval()
    sample = ''
    state = model.init_hidden(1)

    # select a random word id to start sampling
    probs = torch.ones(vocab_size)
    inp = to_var(torch.multinomial(probs, num_samples=1).unsqueeze(1), volatile=True)
    print (inp)
    for i in tqdm_notebook(range(sample_len)):
        output, state = model(inp, state)

        # Sample an id
        probs = output.squeeze().data.exp().cpu()
        word_id = torch.multinomial(probs, 1).numpy()[0]

        # Feed sampled word id to next time step
        inp.data.fill_(word_id)

        # write to file
        word = corpus.vocabulary.index2word[word_id]
        if word == '<EOS>':
            sample += '\n'
        else:
            sample += ' ' + word
    
    return sample

In [45]:
model = LSTM_LM(vocab_size, embed_size=1500, hidden_size=1500, num_layers=2)
model.load_state_dict(torch.load('masnavi-lm.pth'))
model = model.cuda()
sample = get_sample(model, 200)
print(sample)

tensor([[23264]], device='cuda:0')




 کان به نیست بی جوهری
 نیست سر مرگ ما شرع
 مختلف هم حدث از ضلال
 باز سله و بی ندید
 خانه خونابه و زیر نیز
 که آسمانهاست و مردمان
 کای آسمانها از راه
 تا فرو دست دیو
 وقت بازرگان و پهلوان
 نقش یک فرو گشته
 یا فجر و خشک ها
 چون عقلند و می شنید
 تا مگیر و نبیه
 گرچه دیگر سازد چون حیات
 از حدوث و لا سقیم
 از رحمت و در نالیده
 شه خر ز لطف القلوب
 چونک در نیارد در بیم
 زانک اینست بتر سر حق
 کو شدی تا جود و جو
 ای زیان ز سایه و ما
 لقمه بحر اگر یافت
 آدمی چندان چو در نهاد
 پیش اژدرها و سوز تنگ
 کای یزدان بی باغ
 دردسر و شراب کان
 کرده پیمودن ز قهر خویش
 که نعیم و کم ساز
 گاو مرده چه می نمود
 ای صاحب چو می خوردمی
 از قصه هر می آمدی
 این مخنث و می نجست
 معده و شهسواران آن ندید
 در حی که در یافتند
 زندگی در


In [36]:
model = LSTM_LM(vocab_size, embed_size=1500, hidden_size=1500, num_layers=2)
model.load_state_dict(torch.load('lm-masnavi-epoch-38-em-1500-hi-1500-nl-2-1.53-4.63.pth'))
model = model.cuda()

In [37]:
sample = get_sample(model, 300)
print(sample)

tensor([[18976]], device='cuda:0')




 هر دو پیش دین
 تا العارفین بر ترس
 زان به زخم می
 تا شهواتنا آن شرع
 تا اسباب و زیر و کشند
 چار تقوی یک مرد سخت
 وا زراد و ملک مرد
 بود دیگر و دیو و چفت
 گرگ را دهد یک بدان
 ای قلاووزم این مشکل
 از کف خوار چون بهشت
 تا استانید و خانه بود
 همچو دانیش سوی خواب
 که حمال و بانگ گرفت
 دست ندیده می خانه رسید
 صدقوهم و کف و بود
 در حسد و دار و سخن
 که جنبیکم و حیلت و
 ای گبری و ره و
 هست دوزانند از خانه تنگ
 ای اذان از سر به
 در درویشی و تا خاص
 چنگ بگریزد بر گردون
 جوش بگریزد بر بحر
 کز گردونست و پاکی
 بی هوای باز چرخ
 کو حیات رفته ساز
 هاویه و نموده و
 می درمانده بی حال
 پرده ساله و داد
 گر تغب و طاووس
 کای عیسی شده پیش بشیر
 قطره گشته آراست و ریش
 پیش صد ندیدی بی یقین
 بس دیگر گشته از قبور
 لیک رب بود بر زلتست
 لیک یک خفتن و انتقام
 زهر زن پیش ای ستیر
 هرچه چه نگردد چون عزیز
 چون گشادش و هر چه
 باز برگیرم بهر بحر و
 همچو اینجا چو ناری
 وآن حیوان و عار
 کرد هر حالهای در کریم
 ز بحرم و در میگذرد
 سر دیگر کاندرین در غرف
 لیک ما و فعل کاسدست
 قوت و ترکیب و سماع
 سست دان بر آتش حلق
 تا فزونی